Based on [article](https://towardsdatascience.com/pedestrian-tracking-in-real-time-using-yolov3-33439125efdf), [notebook](https://github.com/abhinavsagar/google-colab-notebooks/blob/master/pedestrain_tracking.ipynb) and DeepSort [implementation](https://github.com/ZQPei/deep_sort_pytorch) in Pytorch

In [0]:
import os
from os.path import exists, join, basename

project_name = "deep_sort_pytorch"
if not exists(project_name):
  # clone and install
  !git clone -q --recursive https://github.com/ZQPei/deep_sort_pytorch.git
  
import sys
sys.path.append(project_name)
sys.path.append(join(project_name, 'detector'))
sys.path.append(join(project_name, 'detector/YOLOv3'))

import IPython
from IPython.display import clear_output

In [0]:
if not exists('yolov3.weights'):
  !wget -q https://pjreddie.com/media/files/yolov3.weights
    
if not exists('ckpt.t7'):
  file_id = '1_qwTWdzT9dWNudpusgKavj_4elGgbkUN'
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&id={file_id}" -o ckpt.t7

In [8]:
import cv2
import time

from YOLOv3 import YOLOv3
from deep_sort import DeepSort
from utils.draw import draw_boxes

yolo3 = YOLOv3("deep_sort_pytorch/detector/YOLOv3/cfg/yolo_v3.cfg","yolov3.weights","deep_sort_pytorch/detector/YOLOv3/cfg/coco.names", is_xywh=True)
deepsort = DeepSort("ckpt.t7")

Loading weights from yolov3.weights... Done!
Loading weights from ckpt.t7... Done!


In [9]:
# Download a test video file, convert it from .avi to .mp4 and display it

VIDEO_URL = 'http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi'
DURATION_S = 20  # process only the first 20 seconds

video_file_name = 'video.mp4'
if not exists(video_file_name):
  !wget -q $VIDEO_URL
  dowloaded_file_name = basename(VIDEO_URL)
  # convert to MP4, because we can show only MP4 videos in the colab noteook
  !ffmpeg -y -loglevel info -t $DURATION_S -i $dowloaded_file_name $video_file_name
  
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))
 
clear_output()
video = show_local_mp4_video('video.mp4')
video

Output hidden; open in https://colab.research.google.com to view.

In [0]:
video_capture = cv2.VideoCapture()
if video_capture.open('video.mp4'):
  width, height = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = video_capture.get(cv2.CAP_PROP_FPS)
  !rm -f output.mp4 output.avi
  # can't write out mp4, so try to write into an AVI file
  video_writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
  while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
      break
      
    start = time.time()
    xmin, ymin, xmax, ymax = 0, 0, width, height
    im = frame[ymin:ymax, xmin:xmax, (2,1,0)]
    bbox_xywh, cls_conf, cls_ids = yolo3(im)
    if bbox_xywh is not None:
        mask = cls_ids==0
        bbox_xywh = bbox_xywh[mask]
        bbox_xywh[:,3] *= 1.2
        cls_conf = cls_conf[mask]
        outputs = deepsort.update(bbox_xywh, cls_conf, im)
        if len(outputs) > 0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:,-1]
            frame = draw_boxes(frame, bbox_xyxy, identities, offset=(xmin,ymin))

    end = time.time()
    print("time: {}s, fps: {}".format(end-start, 1/(end-start)))
            
    video_writer.write(frame)
  video_capture.release()
  video_writer.release()
  
  # convert AVI to MP4
  !ffmpeg -y -loglevel info -i output.avi output.mp4
else:
  print("can't open the given input video file!")

In [17]:
show_local_mp4_video('output.mp4', width=960, height=720)

Output hidden; open in https://colab.research.google.com to view.

### Errors Running with official instructions

In [0]:
#!pip install -r deep_sort_pytorch/requirements.txt
#!ls -l deep_sort_pytorch/deep_sort/deep/checkpoint
#!mv yolo_weights_path/yolov3.weights deep_sort_pytorch/detector/YOLOv3/weight/
#!ls -l deep_sort_pytorch/detector/YOLOv3/weight
#!ls -l 

In [0]:
# Download pre-trained YOLO weights
from pathlib import *

deep_sort_path = Path.cwd() / project_name
yolo_weights_path = deep_sort_path / 'detector' / 'YOLOv3' / 'weight'
yolo_weights_file = yolo_weights_path / 'yolov3.weights'
if not exists(yolo_weights_file):
  !wget https://pjreddie.com/media/files/yolov3.weights -O {yolo_weights_file} 

In [0]:
# Download pre-trained Deep Sort weights
deep_sort_weights_path = deep_sort_path / 'deep_sort' / 'deep' / 'checkpoint'
deep_sort_weights_file = deep_sort_weights_path / 'ckpt.t7'

if not exists(deep_sort_weights_file):
  !wget https://drive.google.com/open?id=1_qwTWdzT9dWNudpusgKavj_4elGgbkUN -O {deep_sort_weights_file}

In [0]:
!cd deep_sort_pytorch/detector/YOLOv3/nms && sh build.sh

In [50]:
!python -m pip install darknet

In [51]:
!python deep_sort_pytorch/yolov3_deepsort.py video.mp4 --ignore_display --save_path kdout

Traceback (most recent call last):
  File "deep_sort_pytorch/yolov3_deepsort.py", line 9, in <module>
    from detector import build_detector
  File "/content/deep_sort_pytorch/detector/__init__.py", line 1, in <module>
    from .YOLOv3 import YOLOv3
  File "/content/deep_sort_pytorch/detector/YOLOv3/__init__.py", line 5, in <module>
    from .detector import YOLOv3
  File "/content/deep_sort_pytorch/detector/YOLOv3/detector.py", line 6, in <module>
    from darknet import Darknet
ImportError: cannot import name 'Darknet'
